# Integrating LangChain and Hugging Face

## Install required packages

In [ ]:
%pip install langchain langchain_community
#For API Calls
%pip install huggingface_hub
%pip install transformers
%pip install accelerate
%pip install bitsandbytes

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 3.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
import os

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")


## Approach 1:  Access Models Hosted on Hugging Face Through API

In [5]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}"
)

chain = LLMChain(llm=HuggingFaceHub(repo_id='google/flan-t5-large', 
                                    model_kwargs={'temperature':0, 'max_length':64}),
                                    prompt = prompt)

/tmp/ipykernel_31790/2030021152.py:6: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  chain = LLMChain(llm=HuggingFaceHub(repo_id='google/flan-t5-large',
/home/matteo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_31790/2030021152.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=HuggingFaceHub(repo_id='goog

In [6]:
chain.run("colorful socks")

/tmp/ipykernel_31790/2785530979.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("colorful socks")


AttributeError: 'InferenceClient' object has no attribute 'post'

## Approach 02: Download Model Locally (Create Pipelines)

For this, the use of a GPU is heavily recommended. Intel CPUs are supported, but they will run much slower.

In [7]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

In [9]:
model_id = 'google/flan-t5-large'

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, 
                                              load_in_8bit=True, 
                                              device_map='cpu')

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
None of the available devices `available_devices = None` are supported by the bitsandbytes version you have installed: `bnb_supported_devices = {'cuda', 'xpu', 'hpu', 'npu', '"cpu" (needs an Intel CPU and intel_extension_for_pytorch installed and compatible with the PyTorch version)', 'mps'}`. Please check the docs to see if the backend you intend to use is available and how to install it: https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: None of the available devices `available_devices = None` are supported by the bitsandbytes version you have installed: `bnb_supported_devices = {'cuda', 'xpu', 'hpu', 'npu', '"cpu" (needs an Intel CPU and intel_extension_for_pytorch installed and compatible with the PyTorch version)', 'mps'}`. Please check the docs to see if the backend you intend to use is available and how to install it: https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend